Вынес в отдельный файл. Я когда-то пытался разобраться в том, как устроены генераторы - и это было что-то вроде черновика и полигона.

## Generators and iterators

In [1]:
[i for i in range(9)]

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [2]:
[x for x in range(8)]

[0, 1, 2, 3, 4, 5, 6, 7]

In [3]:
import math

[math.sin(2*x*math.pi) for x in range(10)]

[0.0,
 -2.4492935982947064e-16,
 -4.898587196589413e-16,
 -7.347880794884119e-16,
 -9.797174393178826e-16,
 -1.2246467991473533e-15,
 -1.4695761589768238e-15,
 -1.7145055188062944e-15,
 -1.959434878635765e-15,
 -2.204364238465236e-15]



Квадратные скобки указывают на то, что это список. Внутри квадратных скобок - некая сущность, которая представляет собой формулу или правило получения списка (возможно ли списки без range и прямого указания списка? как минимум должна быть переменная).

[Тут](http://younglinux.info/python/feature/generators) хорошо сказано про это:

> В данном случае конструкция [i for i in range(1,15)] является генератором списка (**может все-таки list comprehensions**?). Вся конструкция заключается в квадратные скобки, что как бы говорит, что будет создан список. Внутри квадратных скобок можно выделить три части: 1) что делаем с элементом (в данном случае ничего не делаем, просто добавляем в список), 2) что берем (в данном случае элемент i), 3) откуда берем (здесь из объекта range). Части отделены друг от друга ключевыми словами for и in.




 Submitted by 51t
on Февраль 23, 2015 - 09:38

Если следовать терминологии, то это всё-таки не генераторы. И в функциональном программировании, и в python генераторы выглядят немного иначе, круглые скобки, а не квадратные:

(i for i in range(5))

А с квадратными - как их только не переводили, но наиболее популярное название на русском - **списочные выражения**

Разница с генераторами в том, что генертатор - это генератор :), а это - это уже готовый список:

In [21]:
>>> [i for i in range(5)]

[0, 1, 2, 3, 4]

In [22]:
>>> (i for i in range(5))

<generator object <genexpr> at 0x7fddd0615510>

если их использовать, как итераторы - то разницы нет:

In [23]:
a = (i for i in range(5))
b = [i for i in range(5)]

In [24]:
for i in a:
    print (i)

0
1
2
3
4


In [25]:

for i in b:
    print (i)

0
1
2
3
4


- дадут абсолютно одинаковый ответ. А разница в том, что первое генерируется по запросу, а второе - это уже полноценный список, который сформирован.

In [26]:
def motor(s):
    print(s)

In [29]:
a = (motor(i) for i in range(5))
a

<generator object <genexpr> at 0x7fddd063a0b0>

In [30]:
b = [motor(i) for i in range(5,10)]
b

5
6
7
8
9


[None, None, None, None, None]

... выдаст 5 6 7 8 9.

Немного более поздние пояснения. И генератор и сокращенный список поддерживают протокол создания итератора через `__iter__` метод. Они "копируют" итератор, потом вызывают от него метод `__next__` или функцию:

    a.next()
    a.next()

это обращение к итератору, отделенному от генератора. Взяты два первых элемента. И теперь:

    for i in a:
        pass

пройдёт уже не 5 раз, а только по трём оставшимся. Если обратиться ещё раз - то не пройдёт ни по какому, потому что элементы - закончились.

Но, собственно, элементов там никаких и нет. Если обратиться к b[0], то он выдаст первый элемент списка. А a[0] - вызовет ошибку, потому что это не список, и пройтись по нему можно только итерацией, и количество элементов в генераторе - неизвестно. Можно превратить генератор в список через a = list(a), тогда a и b будут идентичными списками.

In [31]:
a = range(20)
a

range(0, 20)

In [32]:
z = [i for i in a]
z

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

Но это **еще не генераторы** - это пока просто итераторы.

Генераторы задаются через круглые скобки.

In [33]:
gen = (a for a in range(9))

In [34]:
gen

<generator object <genexpr> at 0x7fddd063aeb0>

In [35]:
for i in gen:
    print(i)

0
1
2
3
4
5
6
7
8


In [36]:
for i in gen:
    print(i)

Второй прогон ничего не выводит - то есть речь идет о генераторе, который хранит внутреннее состояние. Такой штуке не нужно передавать это состояние. Он отрабатывает свой список и засыпает.

In [37]:
hyp = [x*3 for x in range(7)] # здесь все есть сразу и всегда (такой вечнопулемет)
hyp

[0, 3, 6, 9, 12, 15, 18]

In [38]:
hyp_gen = (x*3 for x in range(7)) # здесь в пулемет зарядили ленту от 0 до 6
hyp_gen

<generator object <genexpr> at 0x7fddd06473c0>

In [39]:
for i in hyp:
    print(i)
for i in hyp:
    print(i)

0
3
6
9
12
15
18
0
3
6
9
12
15
18


In [40]:
for j in hyp_gen:
    print(j)
for j in hyp_gen:
    print(j)

0
3
6
9
12
15
18


`yield` - это не сам генератор.  Если это служебное слово использовать вместо `return` в функции - она будет "работать в режиме генератора".

In [41]:
def zum():
    i = range(9)
    return i

a = zum()
print(a)

range(0, 9)


In [42]:
def zumzum():
    i = range(9)
    yield i

a = zumzum()
print(a)

<generator object zumzum at 0x7fddd0647a50>


In [43]:
def zumzum():
    i = range(9)
    yield i

a = zumzum()
print(a)

<generator object zumzum at 0x7fddd0647b30>


In [44]:
def zum():
    i = range(9)
    return i

a = zum()
print(a)

range(0, 9)


In [45]:
a = []

In [46]:
def zum():
    i = range(9)
    return i

a = zum()

for z in a:
    print(a)
for z in a:
    print(a)

range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)
range(0, 9)


In [47]:
def zumzum():
    i = range(9)
    for j in i:
        yield j

a = zumzum()
for z in a:
    print(z)

for z in a:
    print(z)

0
1
2
3
4
5
6
7
8


Разница в следующем:

Функция возвращает результат весь сразу (это пулемет, который выстреливает список одной длинной очередью, я бы даже сказал, одним параллельным залпом), а генератор - по одному значению за раз.

In [48]:
>>> def simple_generator_function():
>>>    yield 1
>>>    yield 2
>>>    yield 3

In [49]:
>>> for value in simple_generator_function():
>>>     print(value)

1
2
3


In [50]:
our_generator = simple_generator_function()

In [51]:
next(our_generator)

1

In [52]:
next(our_generator)

2

In [53]:
next(our_generator)

3

In [54]:
next(our_generator)

StopIteration: 

In [ ]:
# бесконечный генератор
def get_primes(number):
    while True:   # бесконечный цикл
        if is_prime(number): # is_prime внешняя функция - она определяет прайм или нет
            yield number # если да - возвращаем значение
        number += 1 # если нет - или вообще - меняем номер на +1 и повторяем цикл

In [ ]:
# мой вариант бесконечного генератора
def infin():
    number = 1
    while True:
        yield number
        number += 1

In [ ]:
for i in range(10):
    print(next(infin()))

Осечка. ему неоткуда взять следующий номер.

In [ ]:
# мой вариант бесконечного генератора
def infin(number):
    while True:
        yield number
        number += 1

In [ ]:
a = 1
a = infin(a)
for i in range(10):    
    print(next(a))